## สร้างโมเดล Logistic Regression - บทเรียนที่ 4

![ภาพประกอบ Logistic vs. Linear Regression](../../../../../../2-Regression/4-Logistic/images/linear-vs-logistic.png)

#### **[แบบทดสอบก่อนเรียน](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### บทนำ

ในบทเรียนสุดท้ายเกี่ยวกับ Regression ซึ่งเป็นหนึ่งในเทคนิคพื้นฐานของ *คลาสสิก* ML เราจะมาดู Logistic Regression กัน คุณสามารถใช้เทคนิคนี้เพื่อค้นหารูปแบบในการทำนายหมวดหมู่แบบไบนารี ตัวอย่างเช่น ลูกอมนี้เป็นช็อกโกแลตหรือไม่? โรคนี้ติดต่อหรือไม่? ลูกค้าคนนี้จะเลือกสินค้านี้หรือไม่?

ในบทเรียนนี้ คุณจะได้เรียนรู้:

- เทคนิคสำหรับ Logistic Regression

✅ เพิ่มความเข้าใจเกี่ยวกับการทำงานกับ Regression ประเภทนี้ใน [โมดูลการเรียนรู้](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott)

## ความต้องการเบื้องต้น

หลังจากที่เราได้ทำงานกับข้อมูลฟักทองมาแล้ว ตอนนี้เราคุ้นเคยกับมันมากพอที่จะสังเกตเห็นว่ามีหมวดหมู่แบบไบนารีที่เราสามารถทำงานด้วยได้: `Color`

เรามาสร้างโมเดล Logistic Regression เพื่อทำนายว่า *ฟักทองที่กำหนดมีแนวโน้มที่จะมีสีอะไร* (สีส้ม 🎃 หรือสีขาว 👻)

> ทำไมเราถึงพูดถึงการจัดหมวดหมู่แบบไบนารีในบทเรียนที่เกี่ยวกับ Regression? ก็เพื่อความสะดวกทางภาษาเท่านั้น เพราะ Logistic Regression นั้น [จริงๆ แล้วเป็นวิธีการจัดหมวดหมู่](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression) แม้ว่าจะอิงตามเส้นตรงก็ตาม เรียนรู้วิธีอื่นๆ ในการจัดหมวดหมู่ข้อมูลในกลุ่มบทเรียนถัดไป

สำหรับบทเรียนนี้ เราจะต้องใช้แพ็กเกจดังต่อไปนี้:

- `tidyverse`: [tidyverse](https://www.tidyverse.org/) คือ [ชุดของแพ็กเกจ R](https://www.tidyverse.org/packages) ที่ออกแบบมาเพื่อทำให้วิทยาศาสตร์ข้อมูลเร็วขึ้น ง่ายขึ้น และสนุกขึ้น!

- `tidymodels`: [tidymodels](https://www.tidymodels.org/) เป็นกรอบงานที่เป็น [ชุดของแพ็กเกจ](https://www.tidymodels.org/packages/) สำหรับการสร้างโมเดลและการเรียนรู้ของเครื่อง

- `janitor`: [แพ็กเกจ janitor](https://github.com/sfirke/janitor) มีเครื่องมือเล็กๆ ที่เรียบง่ายสำหรับการตรวจสอบและทำความสะอาดข้อมูลที่ไม่สมบูรณ์

- `ggbeeswarm`: [แพ็กเกจ ggbeeswarm](https://github.com/eclarke/ggbeeswarm) มีวิธีการสร้างกราฟแบบ beeswarm โดยใช้ ggplot2

คุณสามารถติดตั้งแพ็กเกจเหล่านี้ได้ด้วยคำสั่ง:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

หรือใช้สคริปต์ด้านล่างเพื่อตรวจสอบว่าคุณมีแพ็กเกจที่จำเป็นสำหรับบทเรียนนี้หรือไม่ และติดตั้งให้ในกรณีที่ยังไม่มี


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **กำหนดคำถาม**

สำหรับจุดประสงค์ของเรา เราจะกำหนดคำถามในรูปแบบไบนารี: 'สีขาว' หรือ 'ไม่ใช่สีขาว' ในชุดข้อมูลของเรายังมีหมวดหมู่ 'ลาย' อยู่ด้วย แต่มีตัวอย่างในหมวดหมู่นี้น้อยมาก ดังนั้นเราจะไม่ใช้มัน และมันจะหายไปเมื่อเราลบค่าที่เป็น null ออกจากชุดข้อมูลอยู่แล้ว

> 🎃 ข้อเท็จจริงสนุกๆ บางครั้งเราจะเรียกฟักทองสีขาวว่า 'ฟักทองผี' ฟักทองเหล่านี้แกะสลักได้ยากกว่า จึงไม่ได้รับความนิยมเท่าฟักทองสีส้ม แต่ก็ดูเท่ดี! ดังนั้นเราสามารถปรับคำถามของเราใหม่ได้ว่า: 'ฟักทองผี' หรือ 'ไม่ใช่ฟักทองผี' 👻

## **เกี่ยวกับการถดถอยโลจิสติก**

การถดถอยโลจิสติกแตกต่างจากการถดถอยเชิงเส้นที่คุณเคยเรียนรู้มาก่อนในหลายๆ แง่มุมที่สำคัญ

#### **การจัดประเภทแบบไบนารี**

การถดถอยโลจิสติกไม่ได้มีคุณสมบัติเหมือนกับการถดถอยเชิงเส้น การถดถอยโลจิสติกให้การคาดการณ์เกี่ยวกับ `หมวดหมู่แบบไบนารี` ("สีส้มหรือไม่ใช่สีส้ม") ในขณะที่การถดถอยเชิงเส้นสามารถคาดการณ์ `ค่าต่อเนื่อง` ได้ เช่น จากแหล่งที่มาของฟักทองและเวลาที่เก็บเกี่ยว *ราคาของมันจะเพิ่มขึ้นเท่าไร*

![อินโฟกราฟิกโดย Dasani Madipalli](../../../../../../2-Regression/4-Logistic/images/pumpkin-classifier.png)

### การจัดประเภทอื่นๆ

ยังมีการถดถอยโลจิสติกประเภทอื่นๆ เช่น มัลติโนเมียลและออร์ดินัล:

- **มัลติโนเมียล** ซึ่งเกี่ยวข้องกับการมีมากกว่าหนึ่งหมวดหมู่ - "สีส้ม, สีขาว, และลาย"

- **ออร์ดินัล** ซึ่งเกี่ยวข้องกับหมวดหมู่ที่มีลำดับ เหมาะสมหากเราต้องการจัดลำดับผลลัพธ์อย่างมีตรรกะ เช่น ฟักทองของเราที่จัดลำดับตามขนาดที่มีจำนวนจำกัด (เล็กมาก, เล็ก, กลาง, ใหญ่, ใหญ่มาก, ใหญ่ที่สุด)

![การถดถอยมัลติโนเมียล vs ออร์ดินัล](../../../../../../2-Regression/4-Logistic/images/multinomial-vs-ordinal.png)

#### **ตัวแปรไม่จำเป็นต้องมีความสัมพันธ์กัน**

จำได้ไหมว่าการถดถอยเชิงเส้นทำงานได้ดีขึ้นเมื่อมีตัวแปรที่มีความสัมพันธ์กันมากขึ้น? การถดถอยโลจิสติกตรงกันข้าม - ตัวแปรไม่จำเป็นต้องสอดคล้องกัน ซึ่งเหมาะกับข้อมูลนี้ที่มีความสัมพันธ์ค่อนข้างอ่อน

#### **คุณต้องการข้อมูลที่สะอาดและมากพอ**

การถดถอยโลจิสติกจะให้ผลลัพธ์ที่แม่นยำมากขึ้นหากคุณใช้ข้อมูลจำนวนมาก ชุดข้อมูลขนาดเล็กของเราไม่เหมาะสมที่สุดสำหรับงานนี้ ดังนั้นโปรดคำนึงถึงข้อนี้

✅ ลองคิดถึงประเภทของข้อมูลที่เหมาะสมกับการถดถอยโลจิสติก

## แบบฝึกหัด - จัดระเบียบข้อมูล

ก่อนอื่น ทำความสะอาดข้อมูลเล็กน้อย โดยลบค่าที่เป็น null และเลือกเฉพาะบางคอลัมน์:

1. เพิ่มโค้ดต่อไปนี้:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


คุณสามารถดูข้อมูลใน dataframe ใหม่ของคุณได้เสมอ โดยใช้ฟังก์ชัน [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html) ดังตัวอย่างด้านล่าง:


In [ ]:
pumpkins_select %>% 
  glimpse()


เรามายืนยันกันว่าเรากำลังทำปัญหาการจำแนกประเภทแบบไบนารีจริงๆ:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### การแสดงผล - แผนภูมิประเภทหมวดหมู่
ตอนนี้คุณได้โหลดข้อมูลฟักทองขึ้นมาอีกครั้งและทำการทำความสะอาดเพื่อให้ได้ชุดข้อมูลที่มีตัวแปรบางตัว รวมถึงตัวแปร Color มาลองแสดงผล dataframe ในโน้ตบุ๊กโดยใช้ไลบรารี ggplot กัน

ไลบรารี ggplot มีวิธีที่น่าสนใจในการแสดงผลข้อมูลของคุณ ตัวอย่างเช่น คุณสามารถเปรียบเทียบการกระจายตัวของข้อมูลสำหรับแต่ละ Variety และ Color ในแผนภูมิประเภทหมวดหมู่ได้

1. สร้างแผนภูมิประเภทนี้โดยใช้ฟังก์ชัน geombar โดยใช้ข้อมูลฟักทองของเรา และกำหนดการจับคู่สีสำหรับแต่ละหมวดหมู่ของฟักทอง (สีส้มหรือสีขาว):


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

จากการสังเกตข้อมูล คุณสามารถเห็นได้ว่า ข้อมูลสีมีความสัมพันธ์กับชนิดพันธุ์อย่างไร

✅ จากกราฟประเภทนี้ มีการสำรวจที่น่าสนใจอะไรบ้างที่คุณสามารถจินตนาการได้?


### การเตรียมข้อมูล: การเข้ารหัสคุณลักษณะ

ชุดข้อมูลฟักทองของเรามีค่าที่เป็นสตริงในทุกคอลัมน์ การทำงานกับข้อมูลประเภทหมวดหมู่เป็นเรื่องง่ายสำหรับมนุษย์ แต่ไม่ใช่สำหรับเครื่องจักร อัลกอริธึมการเรียนรู้ของเครื่องทำงานได้ดีเมื่อใช้ตัวเลข นั่นเป็นเหตุผลว่าทำไมการเข้ารหัสจึงเป็นขั้นตอนสำคัญในกระบวนการเตรียมข้อมูล เนื่องจากช่วยให้เราสามารถเปลี่ยนข้อมูลประเภทหมวดหมู่ให้เป็นข้อมูลเชิงตัวเลขโดยไม่สูญเสียข้อมูล การเข้ารหัสที่ดีนำไปสู่การสร้างโมเดลที่ดี

สำหรับการเข้ารหัสคุณลักษณะ มีตัวเข้ารหัสหลักสองประเภท:

1. **Ordinal encoder**: เหมาะสำหรับตัวแปรประเภทลำดับ (ordinal variables) ซึ่งเป็นตัวแปรหมวดหมู่ที่ข้อมูลมีการเรียงลำดับอย่างมีเหตุผล เช่น คอลัมน์ `item_size` ในชุดข้อมูลของเรา มันจะสร้างการจับคู่ที่แต่ละหมวดหมู่ถูกแทนด้วยตัวเลข ซึ่งตัวเลขนั้นแสดงถึงลำดับของหมวดหมู่ในคอลัมน์

2. **Categorical encoder**: เหมาะสำหรับตัวแปรประเภทนามธรรม (nominal variables) ซึ่งเป็นตัวแปรหมวดหมู่ที่ข้อมูลไม่มีการเรียงลำดับอย่างมีเหตุผล เช่น คุณลักษณะทั้งหมดที่แตกต่างจาก `item_size` ในชุดข้อมูลของเรา มันใช้การเข้ารหัสแบบ one-hot ซึ่งหมายความว่าแต่ละหมวดหมู่จะถูกแทนด้วยคอลัมน์ไบนารี: ตัวแปรที่ถูกเข้ารหัสจะเท่ากับ 1 หากฟักทองนั้นอยู่ใน Variety นั้น และเท่ากับ 0 หากไม่ใช่

Tidymodels มีอีกหนึ่งแพ็กเกจที่น่าสนใจ: [recipes](https://recipes.tidymodels.org/) - แพ็กเกจสำหรับการเตรียมข้อมูล เราจะกำหนด `recipe` ที่ระบุว่าคอลัมน์ตัวทำนายทั้งหมดควรถูกเข้ารหัสเป็นชุดของตัวเลข จากนั้น `prep` เพื่อประมาณค่าปริมาณและสถิติที่จำเป็นสำหรับการดำเนินการใด ๆ และสุดท้าย `bake` เพื่อใช้การคำนวณกับข้อมูลใหม่

> โดยปกติ recipes มักถูกใช้เป็นตัวเตรียมข้อมูลสำหรับการสร้างโมเดล ซึ่งมันจะกำหนดว่าควรมีการดำเนินการใดบ้างกับชุดข้อมูลเพื่อเตรียมให้พร้อมสำหรับการสร้างโมเดล ในกรณีนี้ **แนะนำอย่างยิ่ง** ให้คุณใช้ `workflow()` แทนการประมาณ recipe ด้วย prep และ bake ด้วยตนเอง เราจะเห็นทั้งหมดนี้ในอีกสักครู่
>
> อย่างไรก็ตาม สำหรับตอนนี้ เรากำลังใช้ recipes + prep + bake เพื่อระบุว่าควรมีการดำเนินการใดบ้างกับชุดข้อมูลเพื่อเตรียมให้พร้อมสำหรับการวิเคราะห์ข้อมูล และจากนั้นดึงข้อมูลที่ผ่านการเตรียมพร้อมแล้วพร้อมกับขั้นตอนที่ได้ดำเนินการ


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ ข้อดีของการใช้ ordinal encoder กับคอลัมน์ Item Size คืออะไร?

### วิเคราะห์ความสัมพันธ์ระหว่างตัวแปร

เมื่อเราได้ทำการเตรียมข้อมูลเบื้องต้นแล้ว เราสามารถวิเคราะห์ความสัมพันธ์ระหว่างฟีเจอร์และป้ายกำกับ (label) เพื่อทำความเข้าใจว่าโมเดลจะสามารถทำนายป้ายกำกับจากฟีเจอร์ได้ดีแค่ไหน วิธีที่ดีที่สุดในการวิเคราะห์ประเภทนี้คือการสร้างกราฟ 
เราจะใช้ฟังก์ชัน ggplot geom_boxplot_ อีกครั้ง เพื่อแสดงความสัมพันธ์ระหว่าง Item Size, Variety และ Color ในกราฟแบบหมวดหมู่ (categorical plot) เพื่อให้การแสดงผลข้อมูลดียิ่งขึ้น เราจะใช้คอลัมน์ Item Size ที่ถูกเข้ารหัสแล้ว และคอลัมน์ Variety ที่ยังไม่ได้เข้ารหัส


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### ใช้ swarm plot

เนื่องจาก Color เป็นหมวดหมู่แบบไบนารี (สีขาวหรือไม่ใช่สีขาว) จึงต้องใช้ '[วิธีการเฉพาะทาง](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf)' ในการแสดงผลข้อมูล

ลองใช้ `swarm plot` เพื่อแสดงการกระจายของสีในความสัมพันธ์กับ item_size

เราจะใช้ [ggbeeswarm package](https://github.com/eclarke/ggbeeswarm) ซึ่งมีวิธีการสร้างกราฟแบบ beeswarm โดยใช้ ggplot2 กราฟ beeswarm เป็นวิธีการแสดงจุดข้อมูลที่ปกติจะทับซ้อนกันให้อยู่ข้างกันแทน


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


ตอนนี้เรามีความเข้าใจเกี่ยวกับความสัมพันธ์ระหว่างหมวดหมู่แบบทวิภาคของสีและกลุ่มขนาดที่ใหญ่ขึ้นแล้ว ลองมาสำรวจการใช้โลจิสติกรีเกรสชันเพื่อกำหนดสีที่เป็นไปได้ของฟักทองแต่ละลูกกัน

## สร้างโมเดลของคุณ

เลือกตัวแปรที่คุณต้องการใช้ในโมเดลการจำแนกประเภท และแบ่งข้อมูลออกเป็นชุดฝึกอบรมและชุดทดสอบ [rsample](https://rsample.tidymodels.org/) ซึ่งเป็นแพ็กเกจใน Tidymodels มีโครงสร้างพื้นฐานสำหรับการแบ่งข้อมูลและการสุ่มตัวอย่างซ้ำอย่างมีประสิทธิภาพ:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 เราพร้อมแล้วที่จะฝึกโมเดลโดยการปรับคุณสมบัติการฝึกให้เข้ากับป้ายกำกับการฝึก (สี)

เราจะเริ่มต้นด้วยการสร้างสูตรที่ระบุขั้นตอนการเตรียมข้อมูลที่ควรดำเนินการเพื่อเตรียมข้อมูลให้พร้อมสำหรับการสร้างโมเดล เช่น การเข้ารหัสตัวแปรประเภทให้เป็นชุดของตัวเลข เช่นเดียวกับ `baked_pumpkins` เราสร้าง `pumpkins_recipe` แต่จะไม่ใช้ `prep` และ `bake` เนื่องจากจะถูกรวมไว้ในเวิร์กโฟลว์ ซึ่งคุณจะได้เห็นในอีกไม่กี่ขั้นตอนจากนี้

มีหลายวิธีในการระบุโมเดลการถดถอยโลจิสติกใน Tidymodels ดูที่ `?logistic_reg()` สำหรับตอนนี้ เราจะระบุโมเดลการถดถอยโลจิสติกผ่านเครื่องยนต์เริ่มต้น `stats::glm()`


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


ตอนนี้เรามีสูตรและสเปคของโมเดลแล้ว เราจำเป็นต้องหาวิธีรวมสิ่งเหล่านี้เข้าด้วยกันเป็นวัตถุหนึ่ง ที่จะช่วยจัดการการเตรียมข้อมูล (prep+bake เบื้องหลัง) ฝึกโมเดลด้วยข้อมูลที่ผ่านการเตรียมแล้ว และยังรองรับกิจกรรมการประมวลผลหลังการฝึกโมเดลได้อีกด้วย

ใน Tidymodels วัตถุที่สะดวกนี้เรียกว่า [`workflow`](https://workflows.tidymodels.org/) ซึ่งช่วยจัดเก็บองค์ประกอบการสร้างโมเดลของคุณได้อย่างสะดวกสบาย


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


หลังจากที่กำหนด *workflow* แล้ว สามารถ `train` โมเดลได้โดยใช้ฟังก์ชัน [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html) ฟังก์ชันนี้จะช่วยประเมินสูตรและเตรียมข้อมูลก่อนการฝึกโมเดล ดังนั้นเราไม่จำเป็นต้องทำการเตรียมข้อมูลด้วย prep และ bake ด้วยตัวเอง


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


โมเดลจะแสดงค่าสัมประสิทธิ์ที่ได้จากการฝึกสอนระหว่างการเทรน

ตอนนี้เราได้ฝึกสอนโมเดลด้วยข้อมูลการฝึกสอนเรียบร้อยแล้ว เราสามารถใช้โมเดลนี้เพื่อทำนายผลบนข้อมูลทดสอบได้โดยใช้ [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html) มาเริ่มต้นด้วยการใช้โมเดลเพื่อทำนายป้ายกำกับสำหรับชุดข้อมูลทดสอบ และความน่าจะเป็นของแต่ละป้ายกำกับกัน เมื่อความน่าจะเป็นมากกว่า 0.5 ป้ายกำกับที่ทำนายจะเป็น `WHITE` หากน้อยกว่านั้นจะเป็น `ORANGE`


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


เยี่ยมมาก! นี่ช่วยให้เข้าใจการทำงานของโลจิสติกรีเกรสชันได้ลึกซึ้งยิ่งขึ้น

### เข้าใจได้ดีขึ้นผ่านเมทริกซ์ความสับสน

การเปรียบเทียบแต่ละการทำนายกับค่าจริงที่สอดคล้องกัน ("ground truth") ไม่ใช่วิธีที่มีประสิทธิภาพนักในการประเมินว่ารุ่นทำนายได้ดีแค่ไหน โชคดีที่ Tidymodels มีเครื่องมือเพิ่มเติมที่ช่วยได้: [`yardstick`](https://yardstick.tidymodels.org/) - แพ็กเกจที่ใช้วัดประสิทธิภาพของโมเดลด้วยเมตริกประสิทธิภาพ

หนึ่งในเมตริกประสิทธิภาพที่เกี่ยวข้องกับปัญหาการจำแนกประเภทคือ [`confusion matrix`](https://wikipedia.org/wiki/Confusion_matrix) เมทริกซ์ความสับสนอธิบายว่ารุ่นการจำแนกประเภททำงานได้ดีเพียงใด โดยเมทริกซ์ความสับสนจะจัดทำตารางว่าตัวอย่างในแต่ละคลาสถูกจำแนกอย่างถูกต้องโดยโมเดลกี่ตัวอย่าง ในกรณีของเรา มันจะแสดงให้คุณเห็นว่าฟักทองสีส้มถูกจำแนกเป็นสีส้มกี่ลูก และฟักทองสีขาวถูกจำแนกเป็นสีขาวกี่ลูก นอกจากนี้ เมทริกซ์ความสับสนยังแสดงให้เห็นว่ามีการจำแนกไปยังหมวดหมู่ที่ **ผิด** กี่ตัวอย่างด้วย

ฟังก์ชัน [**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) จาก yardstick ใช้คำนวณการจัดตารางไขว้ระหว่างคลาสที่สังเกตได้และคลาสที่ทำนาย


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


มาทำความเข้าใจเกี่ยวกับ confusion matrix กัน โมเดลของเราถูกขอให้จำแนกฟักทองออกเป็นสองประเภท คือประเภท `white` และประเภท `not-white`

-   หากโมเดลของคุณทำนายว่าฟักทองเป็นสีขาว และในความเป็นจริงมันอยู่ในประเภท 'white' เราเรียกสิ่งนี้ว่า `true positive` ซึ่งแสดงด้วยตัวเลขด้านบนซ้าย

-   หากโมเดลของคุณทำนายว่าฟักทองไม่ใช่สีขาว และในความเป็นจริงมันอยู่ในประเภท 'white' เราเรียกสิ่งนี้ว่า `false negative` ซึ่งแสดงด้วยตัวเลขด้านล่างซ้าย

-   หากโมเดลของคุณทำนายว่าฟักทองเป็นสีขาว และในความเป็นจริงมันอยู่ในประเภท 'not-white' เราเรียกสิ่งนี้ว่า `false positive` ซึ่งแสดงด้วยตัวเลขด้านบนขวา

-   หากโมเดลของคุณทำนายว่าฟักทองไม่ใช่สีขาว และในความเป็นจริงมันอยู่ในประเภท 'not-white' เราเรียกสิ่งนี้ว่า `true negative` ซึ่งแสดงด้วยตัวเลขด้านล่างขวา

| ความจริง |
|:-----:|


|               |        |       |
|---------------|--------|-------|
| **ทำนาย**    | WHITE | ORANGE |
| WHITE         | TP     | FP    |
| ORANGE        | FN     | TN    |

อย่างที่คุณอาจเดาได้ว่า เราต้องการให้มีจำนวน true positive และ true negative มากขึ้น และจำนวน false positive และ false negative น้อยลง ซึ่งหมายความว่าโมเดลทำงานได้ดีขึ้น

confusion matrix มีประโยชน์เพราะมันช่วยให้เราสามารถคำนวณตัวชี้วัดอื่น ๆ ที่ช่วยประเมินประสิทธิภาพของโมเดลการจำแนกประเภทได้ดียิ่งขึ้น มาดูตัวชี้วัดเหล่านี้กัน:

🎓 Precision: `TP/(TP + FP)` หมายถึงสัดส่วนของผลลัพธ์ที่ทำนายว่าเป็นบวกที่เป็นบวกจริง ๆ หรือที่เรียกว่า [positive predictive value](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value")

🎓 Recall: `TP/(TP + FN)` หมายถึงสัดส่วนของผลลัพธ์ที่เป็นบวกจากจำนวนตัวอย่างที่เป็นบวกจริง ๆ หรือที่เรียกว่า `sensitivity`

🎓 Specificity: `TN/(TN + FP)` หมายถึงสัดส่วนของผลลัพธ์ที่เป็นลบจากจำนวนตัวอย่างที่เป็นลบจริง ๆ

🎓 Accuracy: `TP + TN/(TP + TN + FP + FN)` เปอร์เซ็นต์ของป้ายกำกับที่ทำนายได้อย่างถูกต้องสำหรับตัวอย่าง

🎓 F Measure: ค่าเฉลี่ยถ่วงน้ำหนักระหว่าง precision และ recall โดยค่าที่ดีที่สุดคือ 1 และค่าที่แย่ที่สุดคือ 0

มาคำนวณตัวชี้วัดเหล่านี้กัน!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## แสดงกราฟ ROC ของโมเดลนี้

มาทำการแสดงผลอีกครั้งเพื่อดูสิ่งที่เรียกว่า [`กราฟ ROC`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic):


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


กราฟ ROC มักถูกใช้เพื่อดูผลลัพธ์ของตัวจำแนกในแง่ของค่าบวกจริง (True Positives) เทียบกับค่าบวกเท็จ (False Positives) กราฟ ROC โดยทั่วไปจะแสดง `True Positive Rate`/Sensitivity บนแกน Y และ `False Positive Rate`/1-Specificity บนแกน X ดังนั้น ความชันของกราฟและพื้นที่ระหว่างเส้นกลางกับกราฟจึงมีความสำคัญ: คุณต้องการกราฟที่พุ่งขึ้นและข้ามเส้นไปอย่างรวดเร็ว ในกรณีของเรา มีค่าบวกเท็จในช่วงเริ่มต้น และจากนั้นเส้นก็พุ่งขึ้นและข้ามเส้นไปอย่างเหมาะสม

สุดท้ายนี้ เรามาใช้ `yardstick::roc_auc()` เพื่อคำนวณค่าพื้นที่ใต้กราฟ (Area Under the Curve) วิธีหนึ่งในการตีความ AUC คือความน่าจะเป็นที่โมเดลจะจัดอันดับตัวอย่างบวกแบบสุ่มให้สูงกว่าตัวอย่างลบแบบสุ่ม


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


ผลลัพธ์อยู่ที่ประมาณ `0.975` ซึ่งเมื่อพิจารณาว่า AUC มีค่าตั้งแต่ 0 ถึง 1 คุณต้องการคะแนนที่สูง เพราะโมเดลที่ทำนายได้ถูกต้อง 100% จะมีค่า AUC เท่ากับ 1; ในกรณีนี้ โมเดลถือว่า *ค่อนข้างดี*

ในบทเรียนอนาคตเกี่ยวกับการจำแนกประเภท คุณจะได้เรียนรู้วิธีปรับปรุงคะแนนของโมเดล (เช่น การจัดการกับข้อมูลที่ไม่สมดุลในกรณีนี้)

## 🚀ความท้าทาย

ยังมีอีกมากมายเกี่ยวกับการวิเคราะห์ Logistic Regression! แต่วิธีที่ดีที่สุดในการเรียนรู้คือการทดลอง ค้นหาชุดข้อมูลที่เหมาะสมกับการวิเคราะห์ประเภทนี้และสร้างโมเดลด้วยชุดข้อมูลนั้น คุณได้เรียนรู้อะไร? เคล็ดลับ: ลองดู [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) สำหรับชุดข้อมูลที่น่าสนใจ

## ทบทวนและศึกษาด้วยตนเอง

อ่านหน้าแรก ๆ ของ [เอกสารนี้จาก Stanford](https://web.stanford.edu/~jurafsky/slp3/5.pdf) เกี่ยวกับการใช้งาน Logistic Regression ในทางปฏิบัติ ลองคิดถึงงานที่เหมาะสมกับการวิเคราะห์แบบ Regression ประเภทต่าง ๆ ที่เราได้ศึกษาไปจนถึงตอนนี้ งานแบบไหนที่เหมาะสมที่สุด?



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่แม่นยำ เอกสารต้นฉบับในภาษาดั้งเดิมควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ แนะนำให้ใช้บริการแปลภาษาจากผู้เชี่ยวชาญ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่ผิดพลาดซึ่งเกิดจากการใช้การแปลนี้
